In [ ]:
def experiment_generator(saturated_spend, ci, noise):
    '''
    Simulates marketing experiments by calculating sales at different points on the response curve, applying random noise, and returning the results in a DataFrame.

    Args:
        saturated_spend (numpy.ndarray): Array of scaled marketing spend values.
        ci (int): Index to select the appropriate coefficient from `betas_scaled`.
        noise (float): The magnitude of random noise to be applied, where the noise will be a random percentage between -noise and +noise of the calculated sales.

    Returns:
        pandas.DataFrame: A DataFrame containing the results from the experiments.
    '''
    
    # Calculate spend at different points on the response curve
    spend_low = saturated_spend.min()
    spend_med = saturated_spend.mean()
    spend_high = saturated_spend.max()
    
    # Calculate channel contribution (sales) at different points on the response curve
    sales_low = spend_low * betas_scaled[ci]
    sales_med = spend_med * betas_scaled[ci]
    sales_high = spend_high * betas_scaled[ci]
    
    # Generate random noise
    noise_low = np.random.uniform(-noise, noise) * sales_low
    noise_med = np.random.uniform(-noise, noise) * sales_med
    noise_high = np.random.uniform(-noise, noise) * sales_high
    
    # Apply noise to sales
    experiment_results = [
        sales_low + noise_low,
        sales_med + noise_med,
        sales_high + noise_high
    ]
    
    # Create a DataFrame with the results
    data = {
        'experiment_id': ['low', 'med', 'high'],
        'sales': experiment_results,
        'spend': [spend_low, spend_med, spend_high]
    }
    
    df = pd.DataFrame(data)
    
    return df

In [ ]:
mmm_default.plot_channel_contributions_grid(start=0, stop=1.5, num=12);

In [ ]:
mmm_default.plot_channel_contributions_grid(start=0, stop=1.5, num=12, absolute_xrange=True);

In [ ]:
np.random.seed(10)

experiments_social = experiment_generator(df['social_saturated'], 1, 0.25)

experiments_social

In [ ]:
plt.figure(figsize=(8, 5))

sns.lineplot(x=df['social_saturated'], y=df['social_sales'] / df["sales"].max(), label="True response", color="green")
sns.scatterplot(x=experiments_social['spend'], y=experiments_social['sales'], label="Experimental results", color="green", marker="x")

plt.title('Social Experiments', fontsize=16)
plt.xlabel('Saturated spend', fontsize=12)
plt.ylabel('Scaled contribution', fontsize=12)
plt.legend()
plt.show()

In [ ]:
experiments_social

In [ ]:
df_lift_test = pd.DataFrame({
    "channel": ["social_spend_raw", "social_spend_raw", "social_spend_raw"],
    "x": [0, 0, 0],
    "delta_x": experiments_social["spend"] * df["social_spend_raw"].max(),
#    "delta_x": experiments_social["spend"],
    "delta_y": experiments_social["sales"] * df["sales"].max(),
#    "delta_y": experiments_social["sales"],
    "sigma": [0.01, 0.01, 0.01],
    }
)

df_lift_test

In [ ]:
mu = df_lift_test.loc[0, "delta_y"]
sigma = df_lift_test.loc[0, "sigma"]

beta = mu / sigma**2
alpha = mu * beta

x2 = np.linspace(0, 0.2, 1000)
y2 = gamma.pdf(x2, a=alpha, scale=1/beta)

plt.figure(figsize=(8, 6))
plt.plot(x2, y2, 'b-')
plt.fill_between(x2, y2, alpha=0.2, color='blue')
plt.title('Social Experiment 01: Gamma Distribution')
plt.xlabel('delta_y')
plt.ylabel('Probability density')
plt.grid(True)
plt.show()

In [ ]:
mmm_default.add_lift_test_measurements(df_lift_test)
mmm_default.fit(X[train_idx], y[train_idx])

In [ ]:
mmm_default.idata["sample_stats"]["diverging"].sum().item()

In [ ]:
_ = az.plot_trace(
    data=mmm_default.fit_result,
    var_names=[
        "intercept",
        "y_sigma",
        "saturation_beta",
        "saturation_lam",
        "adstock_alpha",
        "gamma_fourier",
    ],
    compact=True,
    backend_kwargs={"figsize": (12, 10), "layout": "constrained"},
)
plt.gcf().suptitle("Model Trace", fontsize=16);

In [ ]:
fig = mmm_default.plot_channel_parameter(param_name="adstock_alpha", figsize=(9, 5))
ax = fig.axes[0]
ax.axvline(x=adstock_alphas[0], color="C0", linestyle="--", label=r"$\alpha_1$")
ax.axvline(x=adstock_alphas[1], color="C1", linestyle="--", label=r"$\alpha_2$")
ax.axvline(x=adstock_alphas[2], color="C2", linestyle="--", label=r"$\alpha_3$")
ax.legend(loc="upper right");

In [ ]:
fig = mmm_default.plot_channel_parameter(param_name="saturation_lam", figsize=(9, 5))
ax = fig.axes[0]
ax.axvline(x=saturation_lamdas[0], color="C0", linestyle="--", label=r"$\lambda_1$")
ax.axvline(x=saturation_lamdas[1], color="C1", linestyle="--", label=r"$\lambda_2$")
ax.axvline(x=saturation_lamdas[2], color="C2", linestyle="--", label=r"$\lambda_3$")
ax.legend(loc="upper right");

In [ ]:
fig = mmm_default.plot_channel_parameter(param_name="saturation_beta", figsize=(9, 5))
ax = fig.axes[0]
ax.axvline(x=betas_scaled[0], color="C0", linestyle="--", label=r"$\beta_1$")
ax.axvline(x=betas_scaled[1], color="C1", linestyle="--", label=r"$\beta_2$")
ax.axvline(x=betas_scaled[2], color="C2", linestyle="--", label=r"$\beta_3$")
ax.legend(loc="upper right");